In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train.csv')

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182080 entries, 0 to 182079
Data columns (total 16 columns):
id                                              182080 non-null object
teacher_id                                      182080 non-null object
teacher_prefix                                  182076 non-null object
school_state                                    182080 non-null object
project_submitted_datetime                      182080 non-null object
project_grade_category                          182080 non-null object
project_subject_categories                      182080 non-null object
project_subject_subcategories                   182080 non-null object
project_title                                   182080 non-null object
project_essay_1                                 182080 non-null object
project_essay_2                                 182080 non-null object
project_essay_3                                 6374 non-null object
project_essay_4                        

In [6]:
train_data.groupby('teacher_prefix').project_is_approved.sum()/train_data.teacher_prefix.value_counts()

Dr.        0.807692
Mr.        0.842022
Mrs.       0.854085
Ms.        0.843052
Teacher    0.794223
dtype: float64

## First, let's look at the distribution of the label

In [5]:
train_data.project_is_approved.value_counts()/train_data.project_is_approved.count()

1    0.847682
0    0.152318
Name: project_is_approved, dtype: float64

In [8]:
train_data.school_state.nunique()

51

## It looks like the distribution of approvals by state is fairly consistent

In [7]:
train_data.groupby('school_state').project_is_approved.sum()/train_data.school_state.value_counts()

AK    0.849192
AL    0.848393
AR    0.832100
AZ    0.841727
CA    0.856431
CO    0.845787
CT    0.871294
DC    0.812639
DE    0.891341
FL    0.824500
GA    0.834087
HI    0.843498
IA    0.846154
ID    0.841869
IL    0.853792
IN    0.847937
KS    0.852830
KY    0.857735
LA    0.834009
MA    0.860138
MD    0.836290
ME    0.858525
MI    0.844478
MN    0.857421
MO    0.857547
MS    0.843384
MT    0.828125
NC    0.853480
ND    0.866935
NE    0.851927
NH    0.867121
NJ    0.843367
NM    0.822052
NV    0.856770
NY    0.853582
OH    0.871467
OK    0.834422
OR    0.855592
PA    0.852543
RI    0.844211
SC    0.864923
SD    0.862348
TN    0.851904
TX    0.815670
UT    0.835110
VA    0.855194
VT    0.848921
WA    0.868050
WI    0.845122
WV    0.867700
WY    0.875706
dtype: float64

In [10]:
train_data.teacher_number_of_previously_posted_projects.value_counts()

0      50067
1      26731
2      17237
3      11893
4       8795
5       6933
6       5617
7       4489
8       3856
9       3253
10      2746
11      2461
12      2237
13      1924
14      1670
15      1549
16      1479
17      1327
18      1226
19      1145
20      1080
21       960
22       892
23       871
25       771
24       761
26       732
27       663
28       600
30       595
       ...  
318        1
386        1
433        1
404        1
324        1
429        1
293        1
406        1
274        1
359        1
309        1
361        1
363        1
236        1
270        1
337        1
365        1
333        1
373        1
422        1
329        1
394        1
326        1
425        1
450        1
392        1
428        1
321        1
320        1
441        1
Name: teacher_number_of_previously_posted_projects, Length: 401, dtype: int64

In [11]:
train_data.project_grade_category.value_counts()

Grades PreK-2    73890
Grades 3-5       61682
Grades 6-8       28197
Grades 9-12      18311
Name: project_grade_category, dtype: int64

In [20]:
train_data.groupby('project_grade_category').project_is_approved.sum()/train_data.project_grade_category.value_counts()

Grades 3-5       0.853977
Grades 6-8       0.843459
Grades 9-12      0.835236
Grades PreK-2    0.847124
dtype: float64

In [13]:
train_data.project_subject_categories.nunique()

51

In [18]:
train_data.project_subject_categories.value_counts()[0:11].sum()/train_data.project_subject_categories.count()

0.81547671353251316

In [19]:
train_data.project_subject_categories.nunique()

51

In [14]:
train_data.teacher_number_of_previously_posted_projects.corr(train_data.project_is_approved)

0.065579438150795713

In [21]:
resource_df = pd.read_csv('resources.csv')

In [22]:
resource_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541272 entries, 0 to 1541271
Data columns (total 4 columns):
id             1541272 non-null object
description    1540980 non-null object
quantity       1541272 non-null int64
price          1541272 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 47.0+ MB


In [23]:
resource_df[0:10]

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95
5,p069063,Last to Finish: A Story About the Smartest Boy...,1,16.99
6,p069063,"Mrs. Gorski, I Think I Have the Wiggle Fidgets...",1,9.95
7,p069063,"See-N-Read 1503905CQ Reading Tool - Book Size,...",2,10.11
8,p096795,"Brewster WPD90218 Wall Pops Flirt Dot, Set of ...",2,9.95
9,p096795,Brewster Wall Pops WPE99065 Peel & Stick Calyp...,2,9.02


In [24]:
resource_df = resource_df.assign(total_price=resource_df.price * resource_df.quantity)

In [25]:
resource_df[0:10]

,id,description,quantity,price,total_price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95,44.85
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59,27.18
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95,74.85
5,p069063,Last to Finish: A Story About the Smartest Boy...,1,16.99,16.99
6,p069063,"Mrs. Gorski, I Think I Have the Wiggle Fidgets...",1,9.95,9.95
7,p069063,"See-N-Read 1503905CQ Reading Tool - Book Size,...",2,10.11,20.22
8,p096795,"Brewster WPD90218 Wall Pops Flirt Dot, Set of ...",2,9.95,19.90
9,p096795,Brewster Wall Pops WPE99065 Peel & Stick Calyp...,2,9.02,18.04


In [27]:
total_price_df = resource_df.groupby('id').total_price.sum().to_frame(name='total_cost')

In [30]:
total_price_df = total_price_df.reset_index()

In [31]:
total_price_df[0:10]

,id,total_cost
0,p000001,833.63
1,p000002,630.28
2,p000003,298.97
3,p000004,1126.22
4,p000005,702.31
5,p000006,130.62
6,p000007,947.88
7,p000008,296.99
8,p000009,335.40
9,p000010,332.40


In [32]:
train_merged = train_data.merge(total_price_df, on='id', how='left')

In [33]:
train_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182080 entries, 0 to 182079
Data columns (total 17 columns):
id                                              182080 non-null object
teacher_id                                      182080 non-null object
teacher_prefix                                  182076 non-null object
school_state                                    182080 non-null object
project_submitted_datetime                      182080 non-null object
project_grade_category                          182080 non-null object
project_subject_categories                      182080 non-null object
project_subject_subcategories                   182080 non-null object
project_title                                   182080 non-null object
project_essay_1                                 182080 non-null object
project_essay_2                                 182080 non-null object
project_essay_3                                 6374 non-null object
project_essay_4                        

In [34]:
train_merged.total_cost.corr(train_merged.project_is_approved)

-0.033018262331184545

In [35]:
train_merged.to_csv('train_with_cost.csv', index=False, index_label=False)

In [36]:
test_data = pd.read_csv('test.csv')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78035 entries, 0 to 78034
Data columns (total 15 columns):
id                                              78035 non-null object
teacher_id                                      78035 non-null object
teacher_prefix                                  78034 non-null object
school_state                                    78035 non-null object
project_submitted_datetime                      78035 non-null object
project_grade_category                          78035 non-null object
project_subject_categories                      78035 non-null object
project_subject_subcategories                   78035 non-null object
project_title                                   78035 non-null object
project_essay_1                                 78035 non-null object
project_essay_2                                 78035 non-null object
project_essay_3                                 2704 non-null object
project_essay_4                                 2704

In [38]:
test_merged = test_data.merge(total_price_df, on='id', how='left')

In [39]:
test_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78035 entries, 0 to 78034
Data columns (total 16 columns):
id                                              78035 non-null object
teacher_id                                      78035 non-null object
teacher_prefix                                  78034 non-null object
school_state                                    78035 non-null object
project_submitted_datetime                      78035 non-null object
project_grade_category                          78035 non-null object
project_subject_categories                      78035 non-null object
project_subject_subcategories                   78035 non-null object
project_title                                   78035 non-null object
project_essay_1                                 78035 non-null object
project_essay_2                                 78035 non-null object
project_essay_3                                 2704 non-null object
project_essay_4                                 2704

In [40]:
test_merged.to_csv('test_with_cost.csv', index=False, index_label=False)

In [41]:
test_reduced = test_merged[['id', 'teacher_number_of_previously_posted_projects', 'total_cost']]

In [42]:
test_reduced.to_csv('test_reduced_with_price.csv', index=False, index_label=False)